## Load model from local to AzureML

In [1]:
import mlflow
from mlflow.deployments import get_deploy_client
import datetime
import json

model_name = 'GradientBoostingRegressor_California'
model_version = 1
model_uri = f'models:/{model_name}/{model_version}'

local_path = mlflow.artifacts.download_artifacts(
    artifact_uri=f"models:/{model_name}/{model_version}"
)

local_path

'/home/vinht/git/mlflow-basic-demo-/mlruns/882174233667255794/models/m-ac4472aa70d34185b8dec9d402e951ad/artifacts'

In [ ]:
mlflow_tracking_uri = 'your-mlflow-uri-on-azureml-workspace'
mlflow.set_tracking_uri(mlflow_tracking_uri)

mlflow.register_model(
    model_uri=f'file://{local_path}', name='GBRegressor'
)

gio: https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize?client_id=04b07795-8ddb-461a-bbee-02f9e1bf7b46&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8400&scope=https%3A%2F%2Fmanagement.azure.com%2F.default+offline_access+openid+profile&state=rhgoLijCTIlxGQPR&code_challenge=qK1FTUAwOWghQrxFwESrZk--mlhY7q-Jr6eg4tF0iW8&code_challenge_method=S256&nonce=4353214bcd92bae65b1cb037bbf6ebb52870105e4af71ce7d1015266e42c0a57&client_info=1&prompt=select_account: Operation not supported
Registered model 'GBRegressor' already exists. Creating a new version of this model...
2025/08/15 12:10:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GBRegressor, version 1
Created version '1' of model 'GBRegressor'.


<ModelVersion: aliases=[], creation_timestamp=1755259822721, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description=None, last_updated_timestamp=1755259822721, metrics=None, model_id=None, name='GBRegressor', params=None, run_id=None, run_link='', source='azureml://artifacts/LocalUpload/250815T121019-65bc2d4c/artifacts', status='READY', status_message=None, tags={}, user_id='', version='1'>

## Create endpoint

In [3]:
deployment_client = get_deploy_client(mlflow.get_tracking_uri())

endpoint_name = 'GBRegressor-' + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_config = {
    "auth_mode": "key",
    "identity": {
        "type": "system_assigned"
    }
}

endpoint_config_path = "endpoint_config.json"
with open(endpoint_config_path, "w") as out:
    out.write(json.dumps(endpoint_config))

In [4]:
endpoint = deployment_client.create_endpoint(
    name=endpoint_name,
    config={"endpoint-config-file": endpoint_config_path},
)

## Create deployment

In [5]:
blue_deployment_name = "blue"

deploy_config = {
    "instance_type": "Standard_F4s_v2",
    "instance_count": 1,
}

deployment_config_path = "deployment_config.json"
with open(deployment_config_path, "w") as outfile:
    outfile.write(json.dumps(deploy_config))

In [6]:
blue_deployment = deployment_client.create_deployment(
    name=blue_deployment_name,
    endpoint=endpoint_name,
    model_uri=f"models:/GBRegressor/1",
    config={"deploy-config-file": deployment_config_path},
)

/home/vinht/miniconda3/envs/my_env/lib/python3.12/site-packages/azureml/mlflow/deploy/_util.py:64: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_model_version_stages`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  if model_stage_or_version in client.get_model_version_stages(None, None):


........................................................................................................................................................................................

In [7]:
traffic_config = {"traffic": {blue_deployment_name: 100}}

traffic_config_path = "traffic_config.json"
with open(traffic_config_path, "w") as outfile:
    outfile.write(json.dumps(traffic_config))

In [8]:
deployment_client.update_endpoint(
    endpoint=endpoint_name,
    config={"endpoint-config-file": traffic_config_path},
)

{'id': '/subscriptions/1d90e0f8-e2bc-4019-aadb-85f8cf30d395/resourceGroups/22127460-rg/providers/Microsoft.MachineLearningServices/workspaces/mlflow-deployment/onlineEndpoints/gbregressor-08151210460811',
 'name': 'gbregressor-08151210460811',
 'type': 'Microsoft.MachineLearningServices/workspaces/onlineEndpoints',
 'systemData': {'createdBy': 'VINH QUÁCH TRẦN QUÁN',
  'createdAt': '2025-08-15T12:10:29.089969Z',
  'lastModifiedAt': '2025-08-15T12:27:24.026753Z'},
 'tags': {},
 'location': 'southeastasia',
 'identity': {'principalId': '69d91899-8d1d-43dd-a28a-22c8102b40af',
  'tenantId': '40127cd4-45f3-49a3-b05d-315a43a9f033',
  'type': 'SystemAssigned'},
 'kind': 'Managed',
 'properties': {'authMode': 'Key',
  'properties': {'azureml.onlineendpointid': '/subscriptions/1d90e0f8-e2bc-4019-aadb-85f8cf30d395/resourcegroups/22127460-rg/providers/microsoft.machinelearningservices/workspaces/mlflow-deployment/onlineendpoints/gbregressor-08151210460811',
   'AzureAsyncOperationUri': 'https://m

## Test endpoint using MLflow `predict()` function

In [9]:
# Read the sample request that's in the JSON file, and then construct a pandas data frame.
with open("test_azure.json", "r") as f:
    sample_request = json.loads(f.read())

deployment_client.predict(endpoint=endpoint_name, df=sample_request['input_data'])

array([0.50518761])